In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import KFold,StratifiedKFold

In [ ]:
train_df = pd.read_csv('input/train.csv')
# df = pd.read_csv('input/train_min.csv')  # small data
test_df = pd.read_csv('input/test.csv')

In [ ]:
X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [ ]:
X_test.head()

In [ ]:
X.head()

In [ ]:
pred_cv = np.zeros(len(X))
pred = np.zeros(len(X_test))

In [ ]:
pred_cv

In [ ]:
pred

In [ ]:
# stacking

models = [
    ('lg', SGDClassifier(loss='log', max_iter=5000, tol=1e-7, alpha=0.3)),
#    ('mlp', MLPClassifier(solver='lbfgs', alpha=0.001, hidden_layer_sizes=(5, 2), random_state=1)),
#    ('rf', RandomForestClassifier(n_estimators=10, criterion='entropy')),
#    ('gnb', GaussianNB()),
#    ('qda', QuadraticDiscriminantAnalysis(tol=1e-12)),
]

cv_out = pd.DataFrame(index=data.iloc[:,0])
test_out = pd.DataFrame(index=test_ids)

# add target to cv_out
cv_out['target'] = data.iloc[:,1].values

for model in models:
    name = model[0]
    cls = model[1]
    print('working on ' + name)
    
    # k-fold
    n_splits = 5
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    pred_cv = np.zeros(len(X))
    pred = np.zeros(len(X_test))
    
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
        x_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        cls.fit(x_train, y_train)
    
        print(cls.predict_proba(X) / n_splits)
        print(cls.predict_proba(X_test) / n_splits)
    
        prev_cv += cls.predict_proba(X) / n_splits
        pred += cls.predict_proba(X_test) / n_splits
    
    print(prev_cv)
        
    cv_out[name] = pred_cv
    test_out[name] = pred

In [ ]:
%debug

In [ ]:
# add statistics
num = len(models);

cv_out['mean'] = cv_out.iloc[:,[1,num]].mean(axis=1)
cv_out['min'] = cv_out.iloc[:,[1,num]].min(axis=1)
cv_out['max'] = cv_out.iloc[:,[1,num]].max(axis=1)

test_out['mean'] = test_out.iloc[:,[0,num-1]].mean(axis=1)
test_out['min'] = test_out.iloc[:,[0,num-1]].min(axis=1)
test_out['max'] = test_out.iloc[:,[0,num-1]].max(axis=1)

In [ ]:
cv_out.head()

In [ ]:
test_out.head()

In [ ]:
cv_out.to_csv('input/train_stack.csv')

In [ ]:
test_out.to_csv('input/test_stack.csv')